# Pipeline Testing

In [1]:
%load_ext autoreload
%autoreload 2

In [37]:
import torch
import pytorch_lightning as pl
import yaml
import importlib
from LightningModules.Embedding.layerless_embedding import LayerlessEmbedding, EmbeddingInferenceCallback
from LightningModules.Filter.vanilla_filter import VanillaFilter, FilterInferenceCallback
from LightningModules.Processing.feature_construction import FeatureStore

# Preprocessing

## Data Loading

In [38]:
with open("LightningModules/Processing/prepare_feature_store.yaml") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

In [39]:
preprocess_dm = FeatureStore(config)

In [40]:
preprocess_dm.prepare_data()

Loading detector...
Detector loaded.
Writing outputs to /mnt/c/Users/Daniel/Dropbox/Research/Publications/ExaTrkX/trackml/feature_store_endcaps
/mnt/c/Users/Daniel/Dropbox/Research/Publications/ExaTrkX/trackml/train_100_events/event000001000
Preparing 1000
Layerless truth graph built for /mnt/c/Users/Daniel/Dropbox/Research/Publications/ExaTrkX/trackml/train_100_events/event000001000 with size (2, 123429)
Building Data for 1000
Augmenting 1000
Loading event /mnt/c/Users/Daniel/Dropbox/Research/Publications/ExaTrkX/trackml/train_100_events/event000001000 with a 0 pT cut
Event 1000 writing graphs to /mnt/c/Users/Daniel/Dropbox/Research/Publications/ExaTrkX/trackml/feature_store_endcaps/1000
/mnt/c/Users/Daniel/Dropbox/Research/Publications/ExaTrkX/trackml/train_100_events/event000001001
Preparing 1001


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/murnanedaniel/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-40-c0135c810ce7>", line 1, in <module>
    preprocess_dm.prepare_data()
  File "/home/murnanedaniel/.local/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py", line 85, in wrapped_fn
    return fn(*args, **kwargs)
  File "/home/murnanedaniel/.local/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py", line 27, in wrapped_fn
    return fn(*args, **kwargs)
  File "/home/murnanedaniel/.local/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py", line 85, in wrapped_fn
    return fn(*args, **kwargs)
  File "/home/murnanedaniel/.local/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py", line 27, in wrapped_fn
    return fn(*args, **kwargs)
  File "/mnt/c/Users/Daniel/Dropbox/Research/Publications/E

TypeError: object of type 'NoneType' has no len()

# Embedding

## Model Loading

In [27]:
with open("LightningModules/Embedding/train_embedding.yaml") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

In [28]:
model = LayerlessEmbedding(config)

## Callback Testing

Add to the callback list any data manipulation methods. For example, EmbeddingInferenceCallback automatically builds the training, validation and testing set for the next stage of the pipeline after training.

In [32]:
callback_list = []

In [29]:
callback_list = [EmbeddingInferenceCallback()]

In [30]:
trainer = pl.Trainer(max_epochs = 1, limit_train_batches=1, limit_val_batches=1, callbacks=callback_list)

GPU available: False, used: False
INFO:lightning:GPU available: False, used: False
TPU available: False, using: 0 TPU cores
INFO:lightning:TPU available: False, using: 0 TPU cores


In [31]:
trainer.fit(model)


  | Name      | Type       | Params
-----------------------------------------
0 | layers    | ModuleList | 1 M   
1 | emb_layer | Linear     | 4 K   
2 | norm      | LayerNorm  | 1 K   
3 | act       | Tanh       | 0     
INFO:lightning:
  | Name      | Type       | Params
-----------------------------------------
0 | layers    | ModuleList | 1 M   
1 | emb_layer | Linear     | 4 K   
2 | norm      | LayerNorm  | 1 K   
3 | act       | Tanh       | 0     


Epoch 0:  50%|█████     | 1/2 [00:05<00:05,  5.94s/it, loss=0.736, v_num=54, train_loss=0.736]
Validating: 0it [00:00, ?it/s]
Epoch 0: 100%|██████████| 2/2 [00:16<00:00,  8.22s/it, loss=0.736, v_num=54, train_loss=0.736, val_loss=0.998, eff=0.829, pur=0.00169]
                                                         

Saving latest checkpoint..
INFO:lightning:Saving latest checkpoint..


Training finished, running inference to build graphs...
Epoch 0: 100%|██████████| 2/2 [06:29<00:00, 194.54s/it, loss=0.736, v_num=54, train_loss=0.736, val_loss=0.998, eff=0.829, pur=0.00169]


1

# Filter

## Model Loading

In [53]:
with open("LightningModules/Filter/train_filter.yaml") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

In [54]:
model = VanillaFilter(config)

## Callback Testing

In [32]:
callback_list = []

In [29]:
callback_list = [FilterInferenceCallback()]

In [55]:
trainer = pl.Trainer(max_epochs = 1, limit_train_batches=1, limit_val_batches=1, callbacks=callback_list)

GPU available: False, used: False
INFO:lightning:GPU available: False, used: False
TPU available: False, using: 0 TPU cores
INFO:lightning:TPU available: False, using: 0 TPU cores


In [ ]:
trainer.fit(model)